In [1]:
run ../../initialize.ipynb

Lax Version : 1.6.0


/project/lgrandi/anaconda3/envs/pax_head/lib/python3.4/site-packages/seaborn-0.8.1-py3.4.egg/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [2]:
from Pre_trigger_peaks import Pre_Trigger
from Pre_trigger_peaks import Primary_Times
#from PI_after_s2 import S2_ionization

In [ ]:
datasets = hax.runs.datasets 
datasets = hax.runs.tags_selection(include=['sciencerun1*'],
                                  exclude=['bad','messy', 'test',
                                           'nofield','lowfield',
                                           'commissioning',
                                           'pmttrip','trip','_pmttrip',
                                           'source_opening',
                                           ],
                                  )
datasets= hax.cuts.selection(datasets, datasets['location'] != '', 'Processed data available')

#Radon
datasets_rn = hax.cuts.selection(datasets, datasets['source__type']=='Rn220', 'Source in place')
dataset_names_rn = datasets_rn['name']

#Bkg
datasets_bkg = hax.cuts.selection(datasets, datasets['source__type']=='none', 'Source in place')
dataset_names_bkg = datasets_bkg['name']

#Krypton
datasets_kr = hax.cuts.selection(datasets, datasets['source__type']=='Kr83m', 'Source in place')
dataset_names_kr = datasets_kr['name']

In [ ]:
dataset_names_bkg.iloc[1000]

In [3]:
df = hax.minitrees.load("170415_1228",
                         treemakers=[Pre_Trigger],
                         preselection=None,
                         force_reload=True,
                                   )

Run 170415_1228: Making Pre_Trigger minitree: 100%|██████████| 19642/19642 [01:56<00:00, 168.19it/s]


In [ ]:
df = hax.minitrees.load("170415_1228",
                         treemakers=[Pre_Trigger, Primary_Times],
                         preselection=None,
                         force_reload=True,
                                   )

### Standard Treatment

In [ ]:
#uncomment aft cut

#Run Number to post-process
run_number="170415_1228"

#Main
pax_config = configuration.load_configuration('XENON1T')
R_tpc = pax_config['DEFAULT']['tpc_radius']

#Load File Location
cache_file_name = '/scratch/midway2/jpienaar/cache_files/'+run_number+ '_Pre_trigger.hdf5'
print (run_number, cache_file_name)
#df = hax.minitrees.load(cache_file = cache_file_name)

#Load AFT Cut Values
with open('/home/jpienaar/SingleElectrons/aft_fit_values.pkl', 'rb') as handle:
    dict_aft_fits = pickle.load(handle)

#Define AFT Cut    
def aft_peak_cut(df):
    aft_means=np.array(dict_aft_fits['Background']['means'])
    aft_sigmas=np.array(dict_aft_fits['Background']['sigmas'])
    bins=dict_aft_fits['Background']['bins']
    df['CutPeakAFT']= True ^ (df.area_fraction_top>np.interp(np.log10(df.area), bins, aft_means-3*aft_sigmas)) ^ (df.area_fraction_top<np.interp(np.log10(df.area), bins, aft_means+3*aft_sigmas)) 
    return df

#Determine Cut Pass/Fail for whole DF
df = aft_peak_cut(df)

#Binning
window_length=5*10**8
t_bins     = np.linspace(0, window_length, 501)
t_bin_width= t_bins[1:]-t_bins[:-1]
r_bins     = np.linspace(0, (R_tpc)**2, 101)
dist_bins  = np.linspace(-R_tpc, R_tpc, 101)
s2_bins    = np.linspace(2, 6, 51)
s2_p_bins  = np.linspace(0, 4, 51)

#Define Blank Hists
livet_histogram=Histdd(bins=[t_bins,
                             s2_bins,
                             ], 
                        axis_names=['delta_T',
                                    's2_area',
                                    ]) 

livet_weights_histogram=Histdd(bins=[t_bins,
                             s2_bins,
                             ], 
                        axis_names=['delta_T',
                                    's2_area',
                                    ]) 

events_histogram=Histdd(bins=[t_bins,
                              s2_bins,
                             ], 
                        axis_names=['delta_T',
                                    's2_area'
                                    ]) 

weights_histogram=Histdd(bins=[t_bins,
                              s2_bins,
                             ], 
                        axis_names=['delta_T',
                                    's2_area'
                                    ]) 


peaks_histogram=Histdd(bins=[t_bins,
                             s2_p_bins,
                             ], 
                        axis_names=['delta_T',
                                    'peak_area'
                                    ]) 


dt_r2_histogram=Histdd(bins=[t_bins,
                             r_bins,
                             ], 
                        axis_names=['delta_T',
                                    'r_dist',
                                    ]) 

weights_dt_r2_histogram=Histdd(bins=[t_bins,
                             r_bins,
                             ], 
                        axis_names=['delta_T',
                                    'r_dist',
                                    ]) 


dt_xy_histogram=Histdd(bins=[t_bins,
                           dist_bins,
                           dist_bins,
                           ],
                    axis_names=['delta_T',
                                 'x_dist',
                                 'y_dist',
                                 ])

#Find all unique primary S2 events
#all_events=pd.unique(df['event_number'].values)
#unique_s2s=[]
#num_s2s=np.min([500000, len(all_events)])
#num_s2s=len(all_events)
#for key, event in basic_minitree.iterrows():
#    unique_s2s.append([event.s2_time, event.x_s2_tpf, event.y_s2_tpf, event.s2])

#Load Minitree with Primary S2 information
primary_minitree = hax.minitrees.load(run_number,
                         treemakers=[Primary_Times, 'Basics'],
                         preselection=None,
                         force_reload=True,
                                   )

#For each unique s2 investigate subsquent S2s
num_s2s=len(primary_minitree)
loop_count=0
for key, event in tqdm(primary_minitree.iterrows()):
    #window_length=window_length
    temp_df = df.loc[(df.global_time>(event.s2_time))&(df.global_time<(event.s2_time+window_length))]
    
    #Find unique events within 1ms
    subsequent_events=pd.unique(temp_df['event_number'].values)
    event_info=[]
    for event_id in subsequent_events:
        event_df=temp_df[temp_df.event_number==event_id].iloc[0]
        event_info.append([np.nanmin([event_df.s1_time, event_df.s2_time]), event_df.event_start])
    
    #Binning Info of events
    bin_allocation_start = np.digitize([x[1]-event.s2_time for x in event_info], bins=t_bins)
    bin_allocation_end = np.digitize([x[0]-event.s2_time for x in event_info], bins=t_bins)
    bin_difference=bin_allocation_end-bin_allocation_start
    
    #Determine Live_time conisdered in each time bin
    start_time=[x[1]-event.s2_time for x in event_info]
    end_time=[np.min([x[0]-event.s2_time, window_length]) for x in event_info]
    
    live_time_array=[]
    #Bin number in digitize of by 1
    for idx, bin_allocation in enumerate(bin_difference):
        if bin_allocation == 0:
            live_time_array.append([t_bins[bin_allocation_start[idx]-1], 
                                    (end_time[idx]-start_time[idx])/t_bin_width[bin_allocation_start[idx]-1]])
        elif bin_allocation == 1:
            live_time_array.append([t_bins[bin_allocation_start[idx]-1], 
                                   (t_bins[bin_allocation_start[idx]]-start_time[idx])/t_bin_width[bin_allocation_start[idx]-1]])
            if bin_allocation_start[idx]<len(t_bins)-1:
                live_time_array.append([t_bins[bin_allocation_start[idx]], 
                                    (end_time[idx]-t_bins[bin_allocation_start[idx]])/t_bin_width[bin_allocation_start[idx]]])
        elif bin_allocation >1:
            live_time_array.append([t_bins[bin_allocation_start[idx]-1], 
                                   (t_bins[bin_allocation_start[idx]]-start_time[idx])/t_bin_width[bin_allocation_start[idx]-1]])
            for index in range(bin_allocation-1):
                if bin_allocation_start[idx]+index<len(t_bins)-1:
                    live_time_array.append([t_bins[bin_allocation_start[idx]+index],1])
            if bin_allocation_start[idx]+(bin_allocation-1)<len(t_bins)-1:
                live_time_array.append([t_bins[bin_allocation_start[idx]+(bin_allocation-1)], 
                                        (end_time[idx]-t_bins[bin_allocation_start[idx]+(bin_allocation-1)])/t_bin_width[bin_allocation_start[idx]+(bin_allocation-1)]])
    
    #Apply AFT Cut
    #temp_df=hax.cuts.selection(temp_df, temp_df['CutPeakAFT'], "CutPeakAFT")
    
    #Some Extra Branches
    r_s2=event.x_s2_tpf**2+event.y_s2_tpf**2
    
    temp_df['x_dist'] = temp_df['x_p_tpf']-event.x_s2_tpf
    temp_df['y_dist'] = temp_df['y_p_tpf']-event.y_s2_tpf
    temp_df['r_dist'] = np.sqrt(temp_df['x_dist']**2+temp_df['y_dist']**2)
    
    temp_df['alpha'] = np.arccos((temp_df['r_dist']**2+r_s2**2-R_tpc**2)/(2*r_s2*temp_df['r_dist']))
    temp_df['area_norm'] = temp_df['alpha'].fillna(np.pi) 
    temp_df['norm'] = temp_df['area_norm']/np.pi
    
    #Binning for bin width rate correction
    peak_bins=np.digitize(temp_df['global_time'].values-event.s2_time, t_bins)
  
    
    #Fill Live_T Histogram
    histogram=Histdd([x[0]+1 for x in live_time_array],
                     [np.log10(event.s2)]*len(live_time_array),
                     weights=[x[1] for x in live_time_array],
                     axis_names=['delta_T',
                                 's2_area',
                                 ], 
                     bins=[t_bins,
                           s2_bins,
                           ])
    livet_histogram+=histogram

    #Fill Live_T Histogram
    histogram=Histdd([x[0]+1 for x in live_time_array],
                     [np.log10(event.s2)]*len(live_time_array),
                     weights=[x[1]**2 for x in live_time_array],
                     axis_names=['delta_T',
                                 's2_area',
                                 ], 
                     bins=[t_bins,
                           s2_bins,
                           ])
    livet_weights_histogram+=histogram

    
    #Fill Events Histogram according to Peak Size
    histogram=Histdd(temp_df['global_time'].values-event.s2_time,
                     np.log10(temp_df['area'].values),
                     weights=1/t_bin_width[peak_bins-1],
                     axis_names=['delta_T',
                                 'peak_area',
                                 ], 
                     bins=[t_bins,
                           s2_p_bins,
                           ])
    peaks_histogram+=histogram
    
    #Fill Events Histogram according to S2 Size
    histogram=Histdd(temp_df['global_time'].values-event.s2_time,
                     [np.log10(event.s2)]*len(temp_df),
                     weights=1/t_bin_width[peak_bins-1],
                     axis_names=['delta_T',
                                 's2_area',
                                 ], 
                     bins=[t_bins,
                           s2_bins,
                           ])
    events_histogram+=histogram
    
    #Fill weights histogram for Events Histogram according to S2 Size
    histogram=Histdd(temp_df['global_time'].values-event.s2_time,
                     [np.log10(event.s2)]*len(temp_df),
                     weights=(1/t_bin_width[peak_bins-1])**2,
                     axis_names=['delta_T',
                                 's2_area',
                                 ], 
                     bins=[t_bins,
                           s2_bins,
                           ])
    weights_histogram+=histogram
            
    ##Fill TR_Dist Histogram
    histogram=Histdd(temp_df['global_time'].values-event.s2_time,
                     (temp_df['r_dist'].values)**2,
                     weights=1/temp_df['norm'].values,
                     axis_names=['delta_T',
                                 'r_dist',
                                 ], 
                     bins=[t_bins,
                           r_bins,
                           ])
    dt_r2_histogram+=histogram
    
    ##Fill Weight TR_Dist Histogram
    histogram=Histdd(temp_df['global_time'].values-event.s2_time,
                     (temp_df['r_dist'].values)**2,
                     weights=(1/temp_df['norm'].values)**2,
                     axis_names=['delta_T',
                                 'r_dist',
                                 ], 
                     bins=[t_bins,
                           r_bins,
                           ])
    weights_dt_r2_histogram+=histogram
    
    ##Fill TXY_Dist Histogram
    histogram=Histdd(temp_df['global_time'].values-event.s2_time,
                     temp_df['x_dist'].values,
                     temp_df['y_dist'].values,
                     axis_names=['delta_T',
                                 'x_dist',
                                 'y_dist',
                                 ], 
                     bins=[t_bins,
                           dist_bins,
                           dist_bins,
                           ])
    dt_xy_histogram+=histogram
    
    loop_count+=1
    if loop_count>5000:
        break
        
#Store in Dict for Later
dict_hist={'version' : 1.0,
           'deltat':events_histogram, 'deltat_weights':weights_histogram,
           'peaks': peaks_histogram, 
           'livet_hist': livet_histogram, 'livet_weights': livet_weights_histogram, 
           'dt_r2':dt_r2_histogram, 'dt_r2_weights':weights_dt_r2_histogram,
           'dt_xy':dt_xy_histogram, 'events': num_s2s}

In [ ]:
df.columns


In [ ]:
plt.figure(figsize=(10, 8))
new_hist=dict_hist['deltat'].sum('s2_area')
new_hist.plot()
plt.yscale('log')

In [ ]:
plt.figure(figsize=(10, 8))
new_hist=dict_hist['livet_hist'].sum('s2_area')
new_hist.plot()
plt.yscale('log')

In [ ]:
plt.figure(figsize=(10, 8))
new_hist=dict_hist['deltat'].sum('s2_area')/dict_hist['livet_hist'].sum('s2_area')*1000000
new_hist.plot()
plt.yscale('log')

### Treemaker Code Test

In [ ]:
import sys
import hax
import numpy as np
from pax import units
from collections import defaultdict
import math

def yield_peak_info(event, event_info):  
    #Get all non-lone-hit peaks in the TPC
    peaks_tmp = []
    for i, peak in enumerate(event.peaks):
        if i == event_info.i_s2 or i == event_info.i_s1  or (peak.n_hits<10 and peak.type!='s1'):
            continue
        else:
            peaks_tmp.append(peak) 
    
    peaks = [p for p in peaks_tmp if p.detector == 'tpc' and p.type!='lone_hit' and p.type!='unknown']
    
    #Ordering by left bound time [10ns]
    peaks = sorted(peaks, key=lambda p: p.left) 
    
    #If peaks list is empty, nothing to do
    if not len(peaks):
        return []
    
    for i, peak in enumerate(peaks):      
        #Make sure event.peak is to the right of S1/S2 by looking at time (ns)
        time_peak = peak.area_midpoint
        if not np.isnan(event_info.time_s1):
            if not time_peak < event_info.time_s1: 
                continue
            else:
                yield peak
        if np.isnan(event_info.time_s1):
            if not time_peak < event_info.time_s2: 
                continue
            else:
                yield peak

class Event_Info:
    def __init__ (self):
        self.x_s2_tpf = np.nan
        self.y_s2_tpf = np.nan
        self.x_s2_nn = np.nan
        self.y_s2_nn = np.nan
        self.time_s2 = np.nan
        self.time_s1 = np.nan
        self.i_s2 = np.nan
        self.i_s1 = np.nan
        self.first_trigger = np.nan

                
class Pre_Trigger(hax.minitrees.MultipleRowExtractor):
    __version__ = '6.0.0'
    uses_arrays=True
    #extra_branches = ['peaks.left', 'peaks.n_hits', 'peaks.area', 'peaks.type',
    #                  'peaks.n_contributing_channels', 'peaks.n_contributing_channels_top',
    #                  'peaks.reconstructed_positions*', 'peaks.area_midpoint']
    extra_branches = ['*']
 
    def extract_data(self, event):       
        results = []
        
        #Initialize Data Structure for storing event level information
        event_info = Event_Info()
        
        #If we have an interaction get both S1 and S2 information
        if len(event.interactions):
            #Get indices of S1/S2 in primary interaction
            event_info.i_s2=event.interactions[0].s2
            event_info.i_s1=event.interactions[0].s1

            #Get time of S1/S2 from start of event
            event_info.time_s1=event.peaks[event_info.i_s1].area_midpoint
            event_info.time_s2=event.peaks[event_info.i_s2].area_midpoint
        #If no interaction use largest S2 greater than 150 PE as "primary S2"
        elif len(event.s2s):
            if event.s2s[0]>150:
                event_info.i_s2 = event.s2s[0]
                
                event_info.time_s2 = event.peaks[event_info.i_s2].area_midpoint

        #Get S2 positions (TPF/NN) of primary S2
        if np.isnan(event_info.i_s2) == False:
            rp_s2 = event.peaks[event_info.i_s2].reconstructed_positions
            for rp in rp_s2:
                if rp.algorithm == 'PosRecTopPatternFit':
                    event_info.x_s2_tpf = rp.x
                    event_info.y_s2_tpf = rp.y
                elif rp.algorithm == 'PosRecNeuralNet':
                    event_info.x_s2_nn = rp.x
                    event_info.y_s2_nn = rp.y
        
        #Identify first peak large enough to be considered a trigger.
        peaks_trigger = [p.area_midpoint for p in event.peaks if p.type=='s1' or p.area>150]
        event_info.first_trigger = min(peaks_trigger, default=1.0e6)   
        
        
        peak_branches = ['area', 'area_fraction_top', 'n_hits', 
                         'n_contributing_channels', 'n_contributing_channels_top']
        for peak in yield_peak_info(event, event_info):
            result = dict({x: getattr(peak, x) for x in peak_branches})
            result['x_s2_tpf'] = event_info.x_s2_tpf
            result['y_s2_tpf'] = event_info.y_s2_tpf
            result['x_s2_nn'] = event_info.x_s2_nn
            result['y_s2_nn'] = event_info.y_s2_nn
            result['global_time'] = peak.area_midpoint + event.start_time
            result['event_stop'] = event.stop_time
            result['event_start'] = event.start_time
            result['s1_time'] = event_info.time_s1+event.start_time
            result['s2_time'] = event_info.time_s2+event.start_time  
            result['p_range_50p_area'] = peak.range_area_decile[5]
            result['p_range_90p_area'] = peak.range_area_decile[9]
            result['rise_time'] = peak.area_decile_from_midpoint[1]
            result['time_before_trigger'] = event_info.first_trigger - peak.area_midpoint
            result['window_length'] = event_info.first_trigger
            for rp in peak.reconstructed_positions:
                if rp.algorithm == 'PosRecTopPatternFit':
                    result['x_p_tpf'] = rp.x
                    result['y_p_tpf'] = rp.y
                    result['xy_gof_tpf'] = rp.goodness_of_fit
                elif rp.algorithm == 'PosRecNeuralNet':
                    result['x_p_nn'] = rp.x
                    result['y_p_nn'] = rp.y
                    result['xy_gof_nn'] = rp.goodness_of_fit        
            if peak.type == 's1': 
                result['type'] = 1
            if peak.type == 's2': 
                result['type'] = 2
            if peak.type == 'lone_hit': 
                result['type'] = 3
            if peak.type == 'unknown': 
                result['type'] = 4   
            results.append(result)
        
        #If no peak objects, still need to store S1/S2 info:
        if not results:
            result = dict({x: np.nan for x in peak_branches})
            result['x_s2_tpf'] = event_info.x_s2_tpf
            result['y_s2_tpf'] = event_info.y_s2_tpf
            result['x_s2_nn'] = event_info.x_s2_nn
            result['y_s2_nn'] = event_info.y_s2_nn
            result['global_time'] = np.nan
            result['event_stop'] = event.stop_time
            result['event_start'] = event.start_time
            result['s1_time'] = event_info.time_s1+event.start_time
            result['s2_time'] = event_info.time_s2+event.start_time  
            result['p_range_50p_area'] = np.nan
            result['p_range_90p_area'] = np.nan
            result['rise_time'] = np.nan
            result['time_before_trigger'] = np.nan
            result['window_length'] = event_info.first_trigger
            result['x_p_tpf'] = np.nan
            result['y_p_tpf'] = np.nan
            result['xy_gof_tpf'] = np.nan
            result['x_p_nn'] = np.nan
            result['y_p_nn'] = np.nan
            result['xy_gof_nn'] = np.nan        
            result['type'] = np.nan   
            results.append(result)
        
        return results
    


In [ ]:
df = hax.minitrees.load("170415_1228",
                         treemakers=[Pre_Trigger],
                         preselection=None,
                         force_reload=True,
                        )

In [ ]:
len(df)

In [ ]:
df.hist('rise_time', bins=100)
plt.yscale('log')

In [ ]:
from pax import units, configuration, datastructure
from collections import defaultdict
import math
from multihist import Histdd, Hist1d
import pickle
from tqdm import tqdm
from Pre_trigger_peaks import Primary_Times

#Run Number to post-process
run_number=dataset=sys.argv[1]

#Init Hax and Studd
hax.init(experiment='XENON1T', 
        pax_version_policy = '6.8.0',
        main_data_paths = ['/project2/lgrandi/xenon1t/processed', '/project/lgrandi/xenon1t/processed'],
        minitree_paths = ['/scratch/midway2/jpienaar/minitrees/',
                         '/project2/lgrandi/xenon1t/minitrees/pax_v6.8.0',
                         '/project/lgrandi/xenon1t/minitrees/pax_v6.8.0',
                         ],
        ) 
    
#Set window to look for previous S2s
window_length=10**9

df=df

#Load Minitree with Primary S2 information
primaries = hax.minitrees.load("170415_1228",
                         treemakers=[Primary_Times, 'Basics'],
                         preselection=None,
                         force_reload=True,
                                   )


pre_existing_fields = []
for field in df.columns:
    pre_existing_fields.append(field)
primary_fields=['x_dist', 'y_dist', 'delay', 's2']


#For each unique s2 investigate subsquent S2s
num_s2s=len(primaries)
loop_count=0
new_df=[]
for key, event in tqdm(df.iterrows()):    
    #window_length=window_length
    #temp_primaries = primaries.loc[(primaries.s2_time<(event.global_time))&(primaries.s2_time>(event.global_time-window_length))]
    primaries['delay'] = event.global_time - primaries.s2_time
    temp_primaries = primaries.loc[(primaries.delay<window_length)&(primaries.delay>0)]
    #print(len(temp_primaries))
    temp_primaries = temp_primaries.sort_values(by=['delay'])
    temp_primaries['x_dist'] = temp_primaries['x_s2_tpf'] - event.x_p_tpf
    temp_primaries['y_dist'] = temp_primaries['y_s2_tpf'] - event.y_p_tpf

    row_entry={}
    
    for field in pre_existing_fields:
        row_entry[field]=event[field]
        
    primary_index=1
    for key, primary in temp_primaries.iterrows():
        for field in primary_fields:
            row_entry['%i_%s' %(primary_index, field)]=primary[field]
        primary_index+=1
    
    new_df.append(row_entry)
    loop_count+=1
    if loop_count>1000:
        break

new_df = pd.DataFrame(new_df)

#new_df.to_hdf('/scratch/midway2/jpienaar/cache_files/%s_Delay.hdf5' %(run_number), key='df') 

#with open('/scratch/midway2/jpienaar/cache_files/%s_dt.pkl' %run_number, 'wb') as handle:
#with open('/scratch/midway2/jpienaar/cache_files/%s.pkl' %run_number, 'wb') as handle:
#    pickle.dump(dict_hist, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
len(new_df)

In [ ]:
new_df.hist('2_delay', bins=100)